In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

#Jets adding
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [2]:
data = pd.read_csv("C:\\Users\\grovenbm\\Documents\\oms_analytics\\CSE 6242\\Project\\Tests\\philly_final_dataset_Reduced.csv")


In [11]:
y = data['Est TOM in Days']
data_Features = data.drop(columns=['Est TOM in Days','sale_month','sale_year','sale_day','avg_days_on_market'])
#data_hot = data_Features.drop(columns=['sale_date'])
#data_hot = data_Features.drop(columns=['sale_date','view_scale','heater_scale','central_air','garage_spaces','has_central_air', 'has_type_heater'])
data_hot = data_Features
data_hot = pd.get_dummies(data_hot, drop_first=True)


In [12]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(data10_hot, y, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(data_hot, y, test_size=0.3, random_state=1)

# Standardize full feature set for Elastic Net variable selection
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train)
X_test_scaled_full = scaler_full.transform(X_test)

In [13]:
# Variable selection with Elastic Net
elasticNet = ElasticNetCV(l1_ratio=0.5, cv=5)
elasticNet.fit(X_train_scaled_full, y_train)

# Get the selected features
coefficients = elasticNet.coef_
selected_features = data_hot.columns[coefficients != 0]
print("Selected features:")
print(selected_features)

Selected features:
Index(['zip_code', 'central_air', 'garage_spaces', 'number_of_bedrooms',
       'number_of_bathrooms', 'number_stories', 'total_area', 'year_built',
       'has_basements', 'sale_price', 'active_inventory', 'new_listings',
       'pending_sales', 'homes_sold', 'sale_to_list_ratio',
       'percent_above_list', 'off_market_2w', 'avg_list_price_per_sqft',
       'avg_listed_price', 'avg_price_per_sqft', 'avg_price_sold',
       'est_list_day', 'est_list_month', 'est_list_year'],
      dtype='object')


In [14]:
print("--------------------------------------End of Variable Selection--------------------------------------")

--------------------------------------End of Variable Selection--------------------------------------


In [15]:
#Instiantiate model
LMmodel = LinearRegression()

#fit model to training data. Using only selected features
X_train_selected = X_train[selected_features]

LMmodel.fit(X_train_selected, y_train)

#predict on test data
x_test_selected = X_test[selected_features]
y_pred = LMmodel.predict(x_test_selected)

print("Predictions:")
print(y_pred)

#calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred)
print(R2)

#Access coefficients
coefficients = LMmodel.coef_
print("Coefficients:")
print(coefficients)

#Access intercept
intercept = LMmodel.intercept_
print("Intercept:")
print(intercept)

Predictions:
[53.76149549 58.4696186  84.53251872 ... 89.78126251 71.23851616
 57.45903742]
RMSE:
18.178997087083324
R2:
0.6083846343725399
Coefficients:
[ 9.41888976e-02 -3.29327978e+00 -4.45163537e+00  2.38468786e-01
  1.51073892e-01  1.24835574e+00 -1.81235008e-07 -7.90761991e-03
 -1.54455485e+00 -2.02090748e-07  2.38223279e-02 -4.09704070e-01
  1.81506501e+00 -1.60990737e+00  2.05525965e+02 -3.38799962e+01
 -6.62717889e+01  6.40060266e-01 -1.52874283e-04 -6.54656112e-01
  1.89248753e-04  2.16153174e-02 -4.48922836e-02 -9.18963715e-01]
Intercept:
-36.80347273832796


In [19]:
#Instantiating Random Forest Model
RFmodel = RandomForestRegressor(n_estimators=50, random_state=1,n_jobs=-1 ,verbose=1)

#RF model fit
RFmodel.fit(X_train, y_train)

#Sanity check
print("Progress...")

#Predicting on test data
y_pred_RF = RFmodel.predict(X_test)
print("Predictions:")
print(y_pred_RF)

# Calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_RF))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred_RF)
print(R2)

# Get feature names
feature_names = X_train.columns

# Access feature importances (since Random Forest models provide feature importance)
print("Feature Importances:")
feature_importance = pd.Series(RFmodel.feature_importances_, index=feature_names)
feature_importance = feature_importance.sort_values(ascending=False)
print(feature_importance)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


Progress...
Predictions:
[47. 49. 77. ... 75. 44. 51.]
RMSE:
0.46487353118886
R2:
0.9997439118269166
Feature Importances:
homes_sold                 4.313126e-01
pending_sales              2.592098e-01
off_market_2w              9.668440e-02
total_area                 7.563232e-02
avg_price_per_sqft         3.609260e-02
new_listings               2.672659e-02
active_inventory           2.443354e-02
sale_to_list_ratio         1.771693e-02
zip_code                   1.582019e-02
percent_above_list         4.929447e-03
avg_list_price_per_sqft    4.578693e-03
avg_price_sold             3.214244e-03
avg_listed_price           2.402260e-03
total_livable_area         4.635287e-04
sale_price                 3.861664e-04
garage_spaces              1.186393e-04
number_stories             9.877637e-05
number_of_bathrooms        8.095813e-05
number_of_bedrooms         7.459844e-05
has_basements              1.745431e-05
est_list_month             3.954511e-06
est_list_year              1.907058e-0

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


In [17]:
# Instantiate Gradient Boosting model

'''
GBmodel = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.15,
    max_depth=4,
    random_state=1,
    verbose=1
)
'''
GBmodel = GradientBoostingRegressor(
    subsample= 1.0, n_estimators= 500,
    min_samples_split= 2, min_samples_leaf= 1,
    max_features= None, max_depth= 4, learning_rate= 0.15
)

# Fit the model on untransformed target
GBmodel.fit(X_train, y_train)

# Predict on test data
y_pred_GB = GBmodel.predict(X_test)

# Evaluate performance
RMSE_GB = np.sqrt(mean_squared_error(y_test, y_pred_GB))
R2_GB = r2_score(y_test, y_pred_GB)

print("Predictions:")
print(y_pred_GB)
print("RMSE (Gradient Boosting):", RMSE_GB)
print("R² (Gradient Boosting):", R2_GB)

# Get feature names
feature_names = X_train.columns

# Access feature importances
print("Feature Importances:")
feature_importance = pd.Series(GBmodel.feature_importances_, index=feature_names)
feature_importance = feature_importance.sort_values(ascending=False)
print(feature_importance)

Predictions:
[46.99875657 49.00089814 76.99705144 ... 74.9935384  44.00393308
 51.00273766]
RMSE (Gradient Boosting): 0.17432797963024993
R² (Gradient Boosting): 0.9999639874331515
Feature Importances (Gradient Boosting):
[9.99267049e-03 1.70170130e-08 5.52535178e-04 1.26403452e-07
 1.25149522e-07 1.42440193e-06 1.31428061e-03 5.19193725e-02
 2.00653370e-04 6.86740669e-11 1.95089159e-06 4.15696403e-02
 8.54679204e-03 2.65114109e-01 4.15315991e-01 1.82077335e-02
 1.05356449e-02 0.00000000e+00 1.12790986e-01 9.74461276e-03
 1.08293153e-02 3.46732474e-02 8.67329763e-03 2.99676831e-08
 1.37646791e-06 1.40670927e-05]


In [18]:
# Define parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiate base model
gbr = GradientBoostingRegressor(random_state=1)

# Set up randomized search
rand_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=50,               # Number of random combinations to try
    scoring='r2',            # Optimize for R² score
    cv=3,                    # 3-fold cross-validation
    verbose=2,
    n_jobs=-1,               # Use all available cores
    random_state=42
)

# Fit to training data
rand_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters Found:")
print(rand_search.best_params_)
print("Best Cross-Validated R² Score:")
print(rand_search.best_score_)

# Evaluate on test set
best_gbr = rand_search.best_estimator_
y_pred_best = best_gbr.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)

print("\nTest Set Evaluation")
print("RMSE:", rmse_best)
print("R²:", r2_best)

#with open("modelTOM.pkl", "wb") as f:
    #import pickle
joblib.dump(best_gbr, 'modelTOM.pkl')

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters Found:
{'subsample': 1.0, 'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 3, 'learning_rate': 0.15}
Best Cross-Validated R² Score:
0.9967867275527623

Test Set Evaluation
RMSE: 0.4844899987014061
R²: 0.9997218433057793


['modelTOM.pkl']